# Screenshot Generator für Dokumentation

Dieses Notebook erstellt die Screenshots für die Projektarbeit:
- `screenshot-forecast-vergleich.png`: Zukunftsprognosen aller 4 Modelle
- `screenshot-backtest-vergleich.png`: Historischer Backtest mit Walk-Forward Validation

In [31]:
import os
import sys
from pathlib import Path

# Projektpfad hinzufügen
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Ordner für Bilder erstellen
images_dir = project_root / "images"
images_dir.mkdir(exist_ok=True)
print(f"Images werden gespeichert in: {images_dir}")

Images werden gespeichert in: /home/daniel/dev/ai/casml4se-stonkswagen/images


In [32]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gw2ml.pipelines.forecast import forecast_item
from gw2ml.data.loaders import load_gw2_series

## Konfiguration

In [33]:
# Mystic Coin Item ID
ITEM_ID = 19976

# Forecast Konfiguration
CONFIG = {
    "data": {
        "days_back": 30,
        "value_column": "buy_unit_price",
    },
    "forecast": {
        "horizon": 12,  # 12 Schritte = 1 Stunde bei 5-Minuten-Intervallen
    },
    "metric": {
        "primary": "mape",
        "metrics": ["mape", "smape", "mae", "rmse"],
    },
    "models": [
        {"name": "ARIMA", "grid": None},
        {"name": "ExponentialSmoothing", "grid": None},
        {"name": "XGBoost", "grid": None},
        {"name": "Chronos2", "grid": None},
    ],
}

# Farbpalette für Modelle
MODEL_COLORS = {
    "ARIMA": "#ff7f0e",            # Orange
    "ExponentialSmoothing": "#2ca02c",  # Grün
    "XGBoost": "#d62728",          # Rot
    "Chronos2": "#9467bd",         # Lila
}
ACTUAL_COLOR = "#1f77b4"  # Blau für tatsächliche Werte

## Forecast generieren

In [34]:
print(f"Generiere Forecasts für Item {ITEM_ID} (Mystic Coin)...")
print(f"Modelle: {[m['name'] for m in CONFIG['models']]}")
print(f"Horizon: {CONFIG['forecast']['horizon']} Schritte (= {CONFIG['forecast']['horizon'] * 5} Minuten)")
print()

result = forecast_item(ITEM_ID, override_config=CONFIG, retrain=False)

models_payload = result.get("models", [])
missing_models = result.get("missing_models", [])

print(f"\nErfolgreich: {[m['model_name'] for m in models_payload]}")
if missing_models:
    print(f"Fehlend: {missing_models}")

Generiere Forecasts für Item 19976 (Mystic Coin)...
Modelle: ['ARIMA', 'ExponentialSmoothing', 'XGBoost', 'Chronos2']
Horizon: 12 Schritte (= 60 Minuten)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/daniel/dev/ai/casml4se-stonkswagen/.venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.




Erfolgreich: ['ARIMA', 'Chronos2', 'ExponentialSmoothing', 'XGBoost']


## Screenshot 1: Forecast-Vergleich (Zukunftsprognosen)

In [35]:
# Daten für Future Forecast sammeln
future_rows = []
for m in models_payload:
    model_name = m.get("model_name")
    f_ts = m.get("future", {}).get("timestamps", [])
    f_vals = m.get("future", {}).get("values", [])
    for ts, val in zip(f_ts, f_vals):
        future_rows.append({"timestamp": ts, "value": val, "model": model_name})

df_future = pd.DataFrame(future_rows)
df_future["timestamp"] = pd.to_datetime(df_future["timestamp"])

# Plot erstellen
fig_future = px.line(
    df_future,
    x="timestamp",
    y="value",
    color="model",
    title=f"Zukunftsprognose: Mystic Coin (nächste {CONFIG['forecast']['horizon'] * 5} Minuten)",
    color_discrete_map=MODEL_COLORS,
    labels={"timestamp": "Zeit", "value": "Preis (Kupfer)", "model": "Modell"},
)

fig_future.update_layout(
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    font=dict(size=14),
    title_font_size=18,
    width=1200,
    height=600,
)

fig_future.show()

In [36]:
# Speichern als PNG
forecast_path = images_dir / "screenshot-forecast-vergleich.png"
fig_future.write_image(str(forecast_path), scale=2)
print(f"Gespeichert: {forecast_path}")

Gespeichert: /home/daniel/dev/ai/casml4se-stonkswagen/images/screenshot-forecast-vergleich.png


## Screenshot 2: Backtest-Vergleich (Walk-Forward Validation)

In [37]:
# Daten für Backtest sammeln
hist_rows = []
actual_rows = None

for m in models_payload:
    model_name = m.get("model_name")
    h_ts = m.get("history", {}).get("timestamps", [])
    h_fc = m.get("history", {}).get("forecast", [])
    h_act = m.get("history", {}).get("actual", [])
    
    # Actual-Werte nur einmal sammeln (sind für alle Modelle gleich)
    if h_ts and h_act and actual_rows is None:
        actual_rows = [{"timestamp": ts, "actual": val} for ts, val in zip(h_ts, h_act) if val is not None]
    
    for ts, val in zip(h_ts, h_fc):
        if val is not None:
            hist_rows.append({"timestamp": ts, "forecast": val, "model": model_name})

df_hist = pd.DataFrame(hist_rows)
df_hist["timestamp"] = pd.to_datetime(df_hist["timestamp"])

if actual_rows:
    df_actual = pd.DataFrame(actual_rows)
    df_actual["timestamp"] = pd.to_datetime(df_actual["timestamp"])

print(f"Backtest Datenpunkte: {len(df_hist)} Prognosen, {len(df_actual) if actual_rows else 0} Actual-Werte")

Backtest Datenpunkte: 6868 Prognosen, 1717 Actual-Werte


In [38]:
# Backtest Plot erstellen
fig_backtest = go.Figure()

# Tatsächlicher Preisverlauf (blau)
if actual_rows:
    fig_backtest.add_trace(go.Scatter(
        x=df_actual["timestamp"],
        y=df_actual["actual"],
        mode="lines",
        name="Tatsächlich",
        line=dict(color=ACTUAL_COLOR, width=2),
    ))

# Prognosen der einzelnen Modelle
for model_name in df_hist["model"].unique():
    df_model = df_hist[df_hist["model"] == model_name]
    fig_backtest.add_trace(go.Scatter(
        x=df_model["timestamp"],
        y=df_model["forecast"],
        mode="lines",
        name=model_name,
        line=dict(color=MODEL_COLORS.get(model_name, "#7f7f7f"), width=1.5, dash="dot"),
    ))

fig_backtest.update_layout(
    title="Backtest: Walk-Forward Validation für Mystic Coin",
    xaxis_title="Zeit",
    yaxis_title="Preis (Kupfer)",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    font=dict(size=14),
    title_font_size=18,
    width=1200,
    height=600,
)

fig_backtest.show()

In [39]:
# Speichern als PNG
backtest_path = images_dir / "screenshot-backtest-vergleich.png"
fig_backtest.write_image(str(backtest_path), scale=2)
print(f"Gespeichert: {backtest_path}")

Gespeichert: /home/daniel/dev/ai/casml4se-stonkswagen/images/screenshot-backtest-vergleich.png


## Metriken-Tabelle

In [40]:
from darts import TimeSeries
from gw2ml.metrics.registry import get_metric

metrics_data = []
for m in models_payload:
    model_name = m.get("model_name")
    h_fc = m.get("history", {}).get("forecast", [])
    h_act = m.get("history", {}).get("actual", [])
    
    row = {"Model": model_name}
    
    if h_fc and h_act and len(h_fc) == len(h_act):
        # NaN-Werte entfernen
        valid_pairs = [(f, a) for f, a in zip(h_fc, h_act) if f is not None and a is not None]
        if valid_pairs:
            fc_vals, act_vals = zip(*valid_pairs)
            ts_actual = TimeSeries.from_values(list(act_vals))
            ts_forecast = TimeSeries.from_values(list(fc_vals))
            
            for metric_name in CONFIG["metric"]["metrics"]:
                try:
                    metric_fn = get_metric(metric_name)
                    value = metric_fn(ts_actual, ts_forecast)
                    if metric_name in ["mape", "smape"]:
                        row[metric_name.upper()] = f"{value:.1f}%"
                    else:
                        row[metric_name.upper()] = f"{value:.1f}"
                except Exception as e:
                    row[metric_name.upper()] = "Error"
        else:
            for metric_name in CONFIG["metric"]["metrics"]:
                row[metric_name.upper()] = "N/A"
    else:
        for metric_name in CONFIG["metric"]["metrics"]:
            row[metric_name.upper()] = "N/A"
    
    metrics_data.append(row)

df_metrics = pd.DataFrame(metrics_data)
print("\n=== Modell-Performance Metriken ===")
print(df_metrics.to_string(index=False))


=== Modell-Performance Metriken ===
               Model MAPE SMAPE   MAE  RMSE
               ARIMA 0.6%  0.6% 129.3 204.6
            Chronos2 0.6%  0.6% 134.2 215.7
ExponentialSmoothing 0.6%  0.6% 124.4 210.8
             XGBoost 0.6%  0.7% 137.7 208.2


## Zusammenfassung

In [41]:
print("\n" + "="*60)
print("GENERIERTE SCREENSHOTS")
print("="*60)

for img_file in images_dir.glob("screenshot-*.png"):
    print(f"  {img_file.name}")

print("\n" + "="*60)
print("METRIKEN FÜR DOKUMENTATION (Tabelle 1)")
print("="*60)
print(df_metrics.to_markdown(index=False))


GENERIERTE SCREENSHOTS
  screenshot-forecast-vergleich.png
  screenshot-backtest-vergleich.png

METRIKEN FÜR DOKUMENTATION (Tabelle 1)
| Model                | MAPE   | SMAPE   |   MAE |   RMSE |
|:---------------------|:-------|:--------|------:|-------:|
| ARIMA                | 0.6%   | 0.6%    | 129.3 |  204.6 |
| Chronos2             | 0.6%   | 0.6%    | 134.2 |  215.7 |
| ExponentialSmoothing | 0.6%   | 0.6%    | 124.4 |  210.8 |
| XGBoost              | 0.6%   | 0.7%    | 137.7 |  208.2 |
